In [3]:
import os
import pandas as pd
import numpy as np
import datetime
import yaml
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime
from datetime import datetime
from datetime import date

import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')


# Google Cloud Language Translation API
# We're using the basic version here == "v2" 
from google.cloud import translate_v2

import timeit



[nltk_data] Downloading package punkt to /Users/euniceliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/euniceliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Major tasks accomplished in this notebook
- 1) use the google cloud api to translate non-English addendums


## Read in Files & Data

In [2]:
## Google Cloud API Credential 
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'sixth-topic-347322-4297c4a3d919.json'

In [29]:
## Addendum with language identified
addendum_untranslated = pd.read_csv("addendum_wdetails_cleaned.csv")
                             

## Make Sure that Google Cloud API Work

In [4]:

# translate_client = translate_v2.Client()

# text = "hola como estas"
# target = "en"

# output = translate_client.translate(text, target_language=target)

# print(output)

{'translatedText': 'Hello how are you', 'detectedSourceLanguage': 'es', 'input': 'hola como estas'}


## Function to Run Translation on Dataset

In [5]:
# def google_translate_messages(one_row):
#     '''
#     Pass in a df row.
#     Find the message under the 'SECTION_DETAILS column'
#     '''
#     one_message = one_row.loc['SECTION_DETAILS']
    
#     # initialize the Google Cloud translation client
#     translate_client = translate_v2.Client()
    
#     # set the target language
#     target = 'en'
    
    
#     try:
#     # apply the translation 
#         output = translate_client.translate(one_message, 
#                                             format_='html',
#                                             target_language=target)
#     except:
#         output = {'translatedText': 'translation_error', 
#                   'detectedSourceLanguage': 'translation_error', 
#                   'orig_content':'translation_error'}

#     return list(output.values())

## Data Manipulation and Cleaning Before Translating

In [30]:
# Check if there is duplicated rows because google translation takes a lot of time
# it is good to prevent running translation twice
# len(addendum_untranslated)
# addendum_untranslated[['SECTION_DETAILS']].drop_duplicates()
# len(addendum_untranslated)


99538

,SECTION_DETAILS
0,Employer will furnish free and convenient cook...
1,Three (3) months experience with references re...
2,Living & laundry facilities available. Housing...
3,Should the Employers worker’s compensation ins...
4,The employer shall provide transportation in t...
...,...
99528,California Tax ID Number: 062-1432-4
99532,Persons seeking employment as experienced Agri...
99534,Other Job Specifications Include:\n1.\tThe wor...
99535,may experience occasional exposure to hazards ...


99538

In [33]:
# Filter out non English Rows

# nonEnglish = addendum_untranslated.loc[(addendum_untranslated['lang1']!="en") | (addendum_untranslated['lang2'].notnull()) | (addendum_untranslated['lang3'].notnull())]
# English = addendum_untranslated.loc[(addendum_untranslated['lang1']=="en") & (addendum_untranslated['lang2'].isnull()) & (addendum_untranslated['lang2'].isnull())]

# assert len(addendum_untranslated)==len(nonEnglish)+len(English)

In [34]:
# Split df into 20 dataframes, so that we can revisit if code breaks + internet crashes 

# split_df = np.array_split(nonEnglish, 10)

# # N rows/columns per df. 
# for i in range(len(split_df)):
#     print('df', i+1, ':', 
#         split_df[i].shape)

df 1 : (1670, 15)
df 2 : (1670, 15)
df 3 : (1670, 15)
df 4 : (1670, 15)
df 5 : (1670, 15)
df 6 : (1670, 15)
df 7 : (1669, 15)
df 8 : (1669, 15)
df 9 : (1669, 15)
df 10 : (1669, 15)


## Translate Job Postings

In [9]:
# Set up to run translation

# output path
# filename = 'translated_msgs_'
# ext = '.csv'


# # For every dataframe in split_df, 
# # run through the translation, unpack the results, 
# # and save as csv

# for i in range(len(split_df)):
#     one_df = split_df[i]
    
#     start_translation_time = timeit.default_timer() #time start
    
#     # run translation
#     one_df['output_list'] = one_df.apply(google_translate_messages, axis = 1)

#     stop_translation_time = timeit.default_timer() #time end
    
#     time_lapse = stop_translation_time - start_translation_time
#     print("took " + str(time_lapse) + " seconds to run")
    
#     # unpack the translation results into their own columns
#     one_df[['translatedText', 'detectedSourceLanguage', 'orig_content']] = \
#         pd.DataFrame(one_df.output_list.to_list(),  
#                      index = one_df.index)

#     one_df.to_csv(filename + str(i) + ext)
    
#     print("wrote results for df ", i + 1)

took 889.24137314 seconds to run
wrote results for df  1
took 928.3444729280002 seconds to run
wrote results for df  2
took 851.3610782659998 seconds to run
wrote results for df  3
took 838.3814368590001 seconds to run
wrote results for df  4
took 848.9665148869999 seconds to run
wrote results for df  5
took 851.0585088010002 seconds to run
wrote results for df  6
took 857.8521529560003 seconds to run
wrote results for df  7
took 912.0228779660001 seconds to run
wrote results for df  8
took 891.6887572940004 seconds to run
wrote results for df  9
took 886.3544101749994 seconds to run
wrote results for df  10


In [35]:
## Read all the translated files

## output path
filename = 'translated_msgs_'
ext = '.csv'

## init first df
translated_msgs_init = pd.read_csv(filename + '0' + ext)
print(translated_msgs_init.shape)
# initialize list to store our pickles
csv = []

## For every file in the folder, read it in as a dataframe, 
## then append to the csv list
## total 10 split datasets 
for i in range(1, len(split_df)):
    df = pd.read_csv( filename + str(i) + ext)
    print(df.shape)
    csv.append(df)

(1670, 20)
(1670, 20)
(1670, 20)
(1670, 20)
(1670, 20)
(1670, 20)
(1669, 20)
(1669, 20)
(1669, 20)
(1669, 20)


In [42]:
## Combine the 10 datasets together
translated_data = pd.concat([translated_msgs_init, *csv])
# Look at the languages detected
# translated_msgs.detectedSourceLanguage.value_counts()
# Check if the combined translated datasets match the number of rows of the original nonEnglish datasets
assert len(translated_data)==len(nonEnglish)

## Merge back with English Job Postings

In [43]:
## Create an indicator for the addendums that are being translated
translated_data['translated'] = 'yes'
## Merge back the Job Posting with purely english
translated_postings = English.append(translated_data, ignore_index=True)
assert len(translated_postings)==len(addendum_untranslated)

In [44]:
translated_postings.head()

,Unnamed: 0,level_0,level_1,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS,is_missing_all,all,lang1,...,lang2,lang2_prob,lang3,lang3_prob,Unnamed: 0.1,output_list,translatedText,detectedSourceLanguage,orig_content,translated
0,0,FOIA_2021-F-05932_FY2020,0,H-300-19274-066174,Meal Provision,E.1,Employer will furnish free and convenient cook...,False,"['en', '0.9999991220220414']",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,FOIA_2021-F-05932_FY2020,1,H-300-19274-066174,Job Requirements,B.6,Three (3) months experience with references re...,False,"['en', '0.9999959646773686']",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,FOIA_2021-F-05932_FY2020,2,H-300-19274-066174,Daily Transportation,F.1,Living & laundry facilities available. Housing...,False,"['en', '0.9999964269685038']",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,FOIA_2021-F-05932_FY2020,3,H-300-19274-066174,Job Requirements,B.6,Should the Employers worker’s compensation ins...,False,"['en', '0.999994357030667']",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,FOIA_2021-F-05932_FY2020,4,H-300-19274-066174,Daily Transportation,F.1,The employer shall provide transportation in t...,False,"['en', '0.9999982081048706']",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
## Clean the columns 
## 1) replace translatedText and orig_content of the original english addendum with Section_Details
translated_postings.orig_content.fillna(translated_postings.SECTION_DETAILS, inplace=True)
translated_postings.translatedText.fillna(translated_postings.SECTION_DETAILS, inplace=True)
## 1) replace translated column that is NAN to "NO"
translated_postings.translated = translated_postings.translated.fillna('No')
## 2) get rid of irrelvant columns
translated_postings_cleaned = translated_postings.loc[:, ~translated_postings.columns.isin(['Unnamed: 0.1', 'output_list',"all","is_missing_all","detectedSourceLanguage"])]
translated_postings_cleaned.head()
# ## 3) write the translated dataset to csv file
translated_postings_cleaned.to_csv('translated_job_postings.csv')


,Unnamed: 0,level_0,level_1,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS,lang1,lang1_prob,lang2,lang2_prob,lang3,lang3_prob,translatedText,orig_content,translated
0,0,FOIA_2021-F-05932_FY2020,0,H-300-19274-066174,Meal Provision,E.1,Employer will furnish free and convenient cook...,en,0.999999,NaN,NaN,NaN,NaN,Employer will furnish free and convenient cook...,Employer will furnish free and convenient cook...,NO
1,1,FOIA_2021-F-05932_FY2020,1,H-300-19274-066174,Job Requirements,B.6,Three (3) months experience with references re...,en,0.999996,NaN,NaN,NaN,NaN,Three (3) months experience with references re...,Three (3) months experience with references re...,NO
2,2,FOIA_2021-F-05932_FY2020,2,H-300-19274-066174,Daily Transportation,F.1,Living & laundry facilities available. Housing...,en,0.999996,NaN,NaN,NaN,NaN,Living & laundry facilities available. Housing...,Living & laundry facilities available. Housing...,NO
3,3,FOIA_2021-F-05932_FY2020,3,H-300-19274-066174,Job Requirements,B.6,Should the Employers worker’s compensation ins...,en,0.999994,NaN,NaN,NaN,NaN,Should the Employers worker’s compensation ins...,Should the Employers worker’s compensation ins...,NO
4,4,FOIA_2021-F-05932_FY2020,4,H-300-19274-066174,Daily Transportation,F.1,The employer shall provide transportation in t...,en,0.999998,NaN,NaN,NaN,NaN,The employer shall provide transportation in t...,The employer shall provide transportation in t...,NO


## Pre-Merging Wrangling with the H-2A Job Certificate Data 

### We need the H-2A Job Certificate Data because we are interested in the case status column that enables us to understand that whether when a employer files an application to hire workers, their cases are approved or not. 

In [8]:
# read in datasets (we are only interested in 2020 and 2021 job certificate data
# because)
df_2020 = pd.read_excel("H_2A_Disclosure_Data_FY2020.xlsx")
df_2021 = pd.read_excel("H_2A_Disclosure_Data_FY2021.xlsx")
# combine into a list
full_data = [df_2020, df_2021]

In [9]:
# find and print column names in 2021 that were not present in 2020
# these columns are not the main variables that we are focusing on
# so it is okay when merging the two years' data to disregard these columns
list(full_data[1].columns.difference(full_data[0].columns))

['ADDENDUM_C_ATTACHED',
 'H-2A_LABOR_CONTRACTOR',
 'HOURLY_SCHEDULE_BEGIN',
 'HOURLY_SCHEDULE_END',
 'MEALS_CHARGE',
 'TOTAL_WORKSITES_RECORDS']

In [10]:
intersect_2021 = list(set(df_2020.columns).intersection(df_2021.columns)) 
h2a_combined_20202021 = pd.concat([df_2020, df_2021]) 
h2a_combined_20202021_int = h2a_combined_20202021[intersect_2021].copy()
# export the data to csv file
h2a_combined_20202021_int.to_csv('h2a_combined_20202021_int.csv')

## Merge the Job Certificate Data with the Job Addendums

In [11]:
h2ajob_certificate=pd.read_csv("h2a_combined_20202021_int.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (95,97,102) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
# how many unique values in the CASE_STATUS column?
h2ajob_certificate['CASE_STATUS'].nunique()
# print the unique values of the CASE_STATUS column 
print(h2ajob_certificate['CASE_STATUS'].unique())

6

['Determination Issued - Certification' 'Determination Issued - Withdrawn'
 'Determination Issued - Denied'
 'Determination Issued - Partial Certification'
 'Determination Issued - Certification (Expired)'
 'Determination Issued - Partial Certification (Expired)']


In [13]:
duplicated_diffoutcomes = h2ajob_certificate.groupby('CASE_NUMBER').CASE_STATUS.nunique() > 1
duplicated_diffoutcomes[duplicated_diffoutcomes].index.tolist()

['H-300-19297-107918',
 'H-300-19310-128739',
 'H-300-19346-194783',
 'H-300-20119-524353',
 'H-300-20234-781846',
 'H-300-20245-799330',
 'H-300-20260-827308',
 'H-300-20324-919674']

In [14]:
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-19297-107918"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-19310-128739"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-19346-194783"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20119-524353"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20234-781846"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20245-799330"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20260-827308"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20324-919674"].CASE_STATUS

13068    Determination Issued - Certification
13069        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

12675    Determination Issued - Certification
24180        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

10841    Determination Issued - Certification
24211        Determination Issued - Withdrawn
24213        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

2925     Determination Issued - Certification
24217        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

156                Determination Issued - Certification
23889    Determination Issued - Certification (Expired)
Name: CASE_STATUS, dtype: object

71       Determination Issued - Certification
24165        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

2        Determination Issued - Certification
23959           Determination Issued - Denied
Name: CASE_STATUS, dtype: object

16592    Determination Issued - Certification
24182        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

In [15]:
h2ajob_certificate_dedup = h2ajob_certificate.drop_duplicates(subset='CASE_NUMBER', keep='first')
## check if the depuplication goes well.. (should print empty set)
afterdedup = h2ajob_certificate_dedup.groupby('CASE_NUMBER').CASE_STATUS.nunique() > 1
afterdedup[afterdedup].index.tolist()
## check the case status category again
h2ajob_certificate_dedup.CASE_STATUS.unique()
## delete the withdrawn category because during classification it is arbitrary that withdrawn is fall into
## certified or denied category
h2ajob_certificate_dedup = h2ajob_certificate_dedup[h2ajob_certificate_dedup['CASE_STATUS'] != 'Determination Issued - Withdrawn']
## check the case status category after deleting withdraw category to make sure the deletion is sucessful 
h2ajob_certificate_dedup.CASE_STATUS.unique()



['H-300-19297-107918',
 'H-300-19310-128739',
 'H-300-19346-194783',
 'H-300-20119-524353',
 'H-300-20234-781846',
 'H-300-20245-799330',
 'H-300-20260-827308',
 'H-300-20324-919674']

array(['Determination Issued - Certification',
       'Determination Issued - Withdrawn',
       'Determination Issued - Denied',
       'Determination Issued - Partial Certification',
       'Determination Issued - Certification (Expired)',
       'Determination Issued - Partial Certification (Expired)'],
      dtype=object)

array(['Determination Issued - Certification',
       'Determination Issued - Denied',
       'Determination Issued - Partial Certification',
       'Determination Issued - Certification (Expired)',
       'Determination Issued - Partial Certification (Expired)'],
      dtype=object)

In [149]:
## read in job data 
translated_job_postings=pd.read_csv("translated_job_postings.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [150]:
## join the job postings of the same case number together
concatenate_job=translated_job_postings.groupby(["CASE_NUMBER"])['translatedText'].apply(lambda x: ''.join(x))
## change to dataframe
concatenate_job_data= concatenate_job.to_frame().rename(columns={'translatedText':'combined_job_postings'}).reset_index()
concatenate_job_data.head()

CASE_NUMBER  \
0  H-300-19274-066154   
1  H-300-19274-066174   
2  H-300-19274-066180   
3  H-300-19274-066199   
4  H-300-19274-066237   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [151]:
## take a look at some cases to make sure the join works well (before join)
pd.set_option('display.max_colwidth', None)
case1=translated_job_postings[translated_job_postings["CASE_NUMBER"]=="H-300-19274-066154"]
case1["translatedText"]

16                                                                                                                                                                                                         Operating large farm tractors and equipment as well as operating a motor vehicle under the influence of drugs is dangerous.  Operator manuals for the equipment have label warnings – Do Not Operate Under the Influence of Drugs.  Work Comp Insurance as well as vehicle insurance companies require safety compliance.  Poor judgment, improper driving, etc. can result in serious and fatal accidents.  Employer has a strict policy prohibiting drug use.\nAssisting with the employers farming operation includes lifting agricultural tools and equipment that weigh up to the lifting requirement listed at B.4.e. \nEmployer may request but cannot require workers to work on their Sabbath. The nature of farm work may require workers to work after dark and possibly weekends depending on weather conditions.

In [152]:
## take a look at some cases to make sure the join works well (before join)
concatenate_job_data.head(1)
## yeah it seems that everything is joined...

CASE_NUMBER  \
0  H-300-19274-066154   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [153]:
## check if case number is job addendums is unique
concatenate_job_data["CASE_NUMBER"].is_unique      
## true >> unique

True

In [154]:
## Finally prepare to merge concatenate_job_data + h2ajob_certificate_dedup
h2ajob_certificate_dedup = h2ajob_certificate_dedup[["CASE_NUMBER","CASE_STATUS"]]
combined_certificate_postings= h2ajob_certificate_dedup.merge(concatenate_job_data, on='CASE_NUMBER', how='inner')
combined_certificate_postings.shape
combined_certificate_postings.head()

(13250, 3)

CASE_NUMBER                           CASE_STATUS  \
0  H-300-20265-835437  Determination Issued - Certification   
1  H-300-20260-827678  Determination Issued - Certification   
2  H-300-20260-827308  Determination Issued - Certification   
3  H-300-20258-821801  Determination Issued - Certification   
4  H-300-20258-821682  Determination Issued - Certification   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [155]:
## export the merged data
combined_certificate_postings.to_csv('combined_certificate_postings.csv')